In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
diarias = pd.read_csv('2_prepared_data/diarias2.csv')

In [17]:
destino = []
ida =[]
numpassagens=[]
qtdiarias=[]
requisicao=[]
solicitante=[]
valor=[]
valtotal=[]
volta=[]

In [5]:
geolocator = Nominatim()

saved_cities = {}
saved_cities["Porto Alegre"]=geolocator.geocode("Porto Alegre")
saved_cities["SANTA ROSA - RS"] = geolocator.geocode("SANTA ROSA - RS") #Pra não apontar pras Filipinas
saved_cities["SANTA ROSA-RS"] = geolocator.geocode("SANTA ROSA - RS") #Pra não apontar pras Filipinas

In [18]:
#PREENCHER VETORES COM BASE NO DATAFRAME
i = 0
def insert_new(data):
    destino.append(data.destino)
    ida.append(data.ida)
    numpassagens.append(data.numpassagens)
    qtdiarias.append(data.qtdiarias)
    requisicao.append(data.requisicao)
    solicitante.append(data.solicitante)
    valtotal.append(data.valtotal)
    valor.append(data.valor)
    volta.append(data.volta)

def update_cur(data):
    last1 = destino[i].split(',')[-1].lstrip().rstrip()
    last2 = data.destino.split(',')[-1].lstrip().rstrip()
    if (last1!=last2):
        destino[i]= destino[i]+", "+data.destino
    numpassagens[i]+=data.numpassagens
    qtdiarias[i]+=data.qtdiarias
    valor[i]+=data.valor
    volta[i]=data.volta
    
insert_new(diarias.iloc[0])

import datetime

for d in range(1,len(diarias)):
    diaria = diarias.iloc[d]
    d1 = datetime.datetime.strptime(diaria.ida,'%Y-%m-%d')
    d2 = datetime.datetime.strptime(volta[i],'%Y-%m-%d')
    if (diaria.requisicao==requisicao[i] and (diaria.ida==ida[i] or ((d1 - d2).days < 2))):
        update_cur(diaria)
    else:
        insert_new(diaria)
        i+=1

In [8]:
def this_is_brasil(city):
    #Se você pegar o código de GARIBALDI-RS, por exemplo,
    #O sistema retorna a pizzaria Di Garibaldi
    #Localizada em Novi Sad,Sérvia
    state = city.split('-')[-1].lstrip().rstrip()
    if (len(state)==2):
        return city+"-BRAZIL"#Frases Dominic Toretto @FrasesToretto
    return city


    
def get_distance(d1,d2):
    d1 = d1.rstrip().lstrip()
    d2 = d2.rstrip().lstrip()
    #Localizar cidades pelo nome
    try:
        if (d1 in saved_cities.keys()):#A menos que a cidade esteja salva
            l1 = saved_cities[d1]
        else:
            l1 = geolocator.geocode(this_is_brasil(d1))
            saved_cities[d1] = l1
        if (d2 in saved_cities.keys()):
            l2 = saved_cities[d2]
        else:
            l2 = geolocator.geocode(this_is_brasil(d2))
            saved_cities[d2]=l2

        #Calcular distância entre cidades
        ll1=(l1.latitude,l1.longitude)
        ll2=(l2.latitude,l2.longitude)
        dist = great_circle(ll1,ll2).km
        if (dist==None):
            return get_distance(d1,d2)
        return dist
    except:
        get_distance(d1,d2)

In [37]:
#PREENCHER VETOR DE DISTÂNCIAS PELO BACKUP SALVO
#Como a conexão irá travar várias vezes devido ao Geocode, é melhor salvar
distancia=[]
with open('distances.txt','r') as distfile:
    lines = distfile.read().split('\n')
    for line in lines:
        distancia.append(float(line))

In [7]:
lastind =len(distancia) #Conexão vai travar uma hora, este é o checkpoint de segurança

In [10]:
#ESTIMAR A DISTÂNCIA PERCORRIDA EM CADA VIAGEM
def filldists(lim):
    global lastind
    global destino
    global distancia
    try:
        lastind =len(distancia) #Conexão vai travar uma hora, este é o checkpoint de segurança
        for i in range(lastind,lim):
            dests = destino[i].split(',')
            dist=0
            for j in range(len(dests)):
                if (j!=0 and j!=len(dests)-1):
                    dist+= get_distance(dests[j-1],dests[j])
                else:
                    if j==0:
                        dist = get_distance("Porto Alegre",dests[0])
                    if (j==len(dests)-1):
                        dist+= get_distance(dests[j],"Porto Alegre")
            distancia.append(dist)
            print(i,dist)
    except:
        filldists(lim)

In [25]:
#SALVAR NO BACKUP
with open('distances.txt','w') as backup:
    for i in range(len(distancia)):
        if (i==0):
            backup.write(str(distancia[i]))
        else:
            backup.write('\n'+str(distancia[i]))

In [19]:
# O PROCESSO DE CALCULAR DISTÂNCIAS É MUITO LONGO
# Descomentar por segurança
#filldists(<até qual index ir>)

#filldists(200)
#filldists(400)
#filldists(600)
#filldists(800)
#filldists(1000)
#filldists(1200)
#filldists(1400)
#filldists(1600)
#filldists(1800)
#filldists(2000)
#filldists(2200)
#filldists(2400)
#filldists(2600)
#filldists(2800)
#filldists(3000)
#filldists(3200)
#filldists(3400)
#filldists(3600)
#filldists(3800)
#filldists(4000)
#filldists(4200)
filldists(len(destino))

0 3239.4732966587826
1 133.36736141833978
2 190.85268388371784
3 3239.4732966587826
4 180.8686498306579
5 133.36736141833978
6 2247.5258357216394
7 502.8588048794046
8 546.024094004526
9 269.78367787186176
10 479.1842937977543
11 881.4737118274179
12 160.98598383381005
13 2247.5258357216394
14 160.98598383381005
15 3396.9324382746026
16 189.14179962621003
17 3239.4732966587826
18 133.36736141833978
19 2247.5258357216394
20 269.78367787186176
21 440.71443501476733
22 160.98598383381005
23 659.7116448635884
24 1048.8959400919016
25 166.82537479658316
26 133.36736141833978
27 225.32616385125993
28 752.369932397836
29 133.36736141833978
30 160.98598383381005
31 183.83104636668043
32 241.6620440265702
33 3239.4732966587826
34 200.8367179367776
35 262.15719517605794
36 213.3056129317447
37 881.4737118274179
38 241.6620440265702
39 863.8497700672084
40 160.98598383381005
41 211.89915659665945
42 1705.5986638779646
43 133.36736141833978
44 318.1130831488723
45 241.6620440265702
46 133.36736141

655 399.03253296287187
656 574.8632095612984
657 724.3130782737989
658 733.2031604492425
659 555.6570104533265
660 241.13179756036502
661 881.4737118274179
662 348.585913371337
663 466.6567397570583
664 212.67177687232882
665 241.6620440265702
666 207.73647540744582
667 547.2270548381385
668 235.6302350766727
669 213.3056129317447
670 509.12178994766543
671 686.2079424796359
672 692.9683028090047
673 574.8632095612984
674 147.7725435047833
675 353.1626648198953
676 399.03253296287187
677 133.3068063577386
678 21026.594381930423
679 727.5700774310571
680 311.43828837278306
681 1129.7615117383477
682 661.5734959925682
683 724.3130782737989
684 200.8367179367776
685 778.0493792876225
686 637.3832550980505
687 399.03253296287187
688 435.1524327997215
689 1705.5986638779646
690 374.203890236604
691 710.8423889667035
692 241.6620440265702
693 169.38550593989314
694 3239.4732966587826
695 456.89656253229674
696 442.6495310538045
697 598.222334484828
698 147.7725435047833
699 582.4600395141606

1260 3239.4732966587826
1261 323.75662599047223
1262 229.32197105121134
1263 578.8816191104605
1264 343.31977956406786
1265 447.21059496841326
1266 376.468449696157
1267 3239.4732966587826
1268 524.592080563426
1269 192.40241506849006
1270 358.8398872107527
1271 160.98598383381005
1272 200.8367179367776
1273 615.8462290809115
1274 338.8449539900399
1275 3239.4732966587826
1276 772.10404963854
1277 365.1817765946124
1278 291.86658853176215
1279 192.5762614630574
1280 242.9031342777094
1281 374.8979320999531
1282 289.6725262065315
1283 377.919448821356
1284 3239.4732966587826
1285 488.9809812399046
1286 200.8367179367776
1287 216.02259068355707
1288 593.202978216784
1289 180.22799930330132
1290 216.02259068355707
1291 220.62250597662404
1292 3239.4732966587826
1293 343.31977956406786
1294 455.50978271610404
1295 166.82537479658316
1296 568.2407238070459
1297 3239.4732966587826
1298 231.09304647017584
1299 142.5132465915932
1300 1705.5986638779646
1301 343.31977956406786
1302 504.50711313

1905 695.5421913195128
1906 1363.4925609664172
1907 242.97970847176572
1908 863.8497700672083
1909 863.8497700672083
1910 3239.4732966587826
1911 1002.0947605810528
1912 1184.9041948719
1913 1428.1839660559774
1914 3239.4732966587826
1915 319.5504491494089
1916 846.2258283069987
1917 881.4737118274179
1918 787.2210073182114
1919 1203.7852848392465
1920 881.4737118274179
1921 796.1580503970606
1922 20508.859278796328
1923 160.98598383381005
1924 881.4737118274179
1925 770.8840098132558
1926 3239.4732966587826
1927 1428.1839660559774
1928 648.105379467506
1929 289.13107310300455
1930 150.42616213947474
1931 160.98598383381005
1932 329.38199465401806
1933 1038.188636359083
1934 365.1817765946124
1935 999.1016775868154
1936 3239.4732966587826
1937 287.5159749973433
1938 234.3068476082081
1939 263.55836928256974
1940 189.14179962621003
1941 150.42616213947474
1942 211.89915659665945
1943 142.3524494666618
1944 287.64055661045546
1945 155.42341879013003
1946 163.39275667167306
1947 228.71513

2654 402.04025718796436
2655 12.798225391519793
2656 663.7799952167034
2657 955.9486500955619
2658 271.5323682262025
2659 237.0025016859584
2660 185.15189194305543
2661 146.4385845082823
2662 926.6037585398828
2663 271.05137763452524
2664 790.9756281443581
2665 278.3126962479063
2666 633.9536724844656
2667 605.5596041151691
2668 967.3837989916467
2669 86.62184956760592
2670 656.7310199582872
2671 30113.03867239327
2672 294.05455410536615
2673 242.17849528313488
2674 328.18354126765996
2675 160.98598383381005
2676 319.5504491494089
2677 133.36736141833978
2678 192.40241506849006
2679 3239.4732966587826
2680 621.105431143402
2681 846.2258283069987
2682 200.8367179367776
2683 241.6620440265702
2684 241.13179756036502
2685 241.6620440265702
2686 502.8588048794046
2687 328.52525365930035
2688 219.8029355439527
2689 637.2896315529786
2690 192.40241506849006
2691 534.138875018517
2692 180.22799930330132
2693 623.6506335365499
2694 160.98598383381005
2695 180.8686498306579
2696 1705.5986638779

3322 588.9821622881094
3323 396.02797426142774
3324 295.6978251997442
3325 448.6720398935498
3326 373.076097790203
3327 15674.832766264735
3328 186.51653336015272
3329 206.80338433374328
3330 276.5501661996766
3331 248.94824772049031
3332 384.2280460778293
3333 970.9230279358087
3334 328.42077791788427
3335 757.4456128800823
3336 213.3056129317447
3337 234.11972215416273
3338 215.8946182192451
3339 207.73647540744582
3340 309.8271861123459
3341 815.7033464253958
3342 377.919448821356
3343 206.80338433374328
3344 188.41908601311138
3345 194.16568660218115
3346 248.2224057713141
3347 364.3847140533845
3348 1137.7204032405882
3349 268.96546045694856
3350 184.19670271705698
3351 202.51555791460186
3352 932.7731700725421
3353 235.6302350766727
3354 224.69925971700803
3355 187.51470272245496
3356 850.771407022728
3357 973.792938163574
3358 3239.4732966587826
3359 617.0546303159708
3360 782.3728062277186
3361 973.792938163574
3362 241.6620440265702
3363 1100.982750359958
3364 696.574084370196

3915 241.6620440265702
3916 554.281369443625
3917 427.31556142939314
3918 319.5504491494089
3919 241.6620440265702
3920 386.9550912569649
3921 3239.4732966587826
3922 323.9560988922972
3923 673.0751738021361
3924 794.8469659424907
3925 768.837595633355
3926 794.8469659424907
3927 206.5616890539784
3928 536.8683825372849
3929 159.75623623267762
3930 327.010023973256
3931 877.8683756551952
3932 241.13179756036502
3933 570.5985359463772
3934 133.36736141833978
3935 3239.4732966587826
3936 1192.4137773692637
3937 3239.4732966587826
3938 852.8339121789148
3939 271.05137763452524
3940 1003.5534138214762
3941 478.7763323436928
3942 804.2852719986212
3943 262.38083362950255
3944 756.7607388142594
3945 840.2951857479981
3946 854.952552929758
3947 313.64003259848255
3948 1175.1736793155997
3949 261.01387537935256
3950 707.9387226494612
3951 576.287056310059
3952 251.84571656837636
3953 977.0844953162698
3954 169.38550593989314
3955 881.4737118274179
3956 621.105431143402
3957 898.7982958470286
3

4654 3239.4732966587826
4655 600.5012870807402
4656 3239.4732966587826
4657 365.1817765946124
4658 411.86010145028183
4659 3239.4732966587826
4660 189.14179962621003
4661 288.2537891745627
4662 267.86109731627914
4663 695.0211758398959
4664 631.5880702892434
4665 502.8588048794046
4666 3239.4732966587826
4667 320.038321998707
4668 623.6506335365499
4669 263.6939063292142
4670 455.50978271610404
4671 462.2757491929495
4672 530.3487522313192
4673 337.18784184795317
4674 450.58706857534276
4675 323.75662599047223
4676 3239.4732966587826
4677 599.9060378295832
4678 16511.83082669638
4679 223.10319012945416
4680 681.9516477111803
4681 442.07214284087684
4682 581.0001324691236
4683 455.50978271610404
4684 568.8519490067167
4685 3239.4732966587826
4686 761.9790242806163
4687 3239.4732966587826
4688 589.7075913736771
4689 300.5931063723216
4690 337.18784184795317
4691 3239.4732966587826
4692 490.8959249303276
4693 623.6506335365499
4694 635.6900118207193
4695 551.6800459800725
4696 3239.473296

In [23]:
#CORRIGIR FORMATAÇÃO DAS DATAS
for i in range(len(ida)):
    date1 = ida[i]
    date2 = volta[i]
    
    ds1 = date1.split('/')
    ds2 = date2.split('/')
    if (len(ds1)==3):
        ida[i]=str(ds1[2])+'-'+str(ds1[1])+'-'+str(ds1[0])
    if (len(ds2)==3):
        volta[i]=str(ds2[2])+'-'+str(ds2[1])+'-'+str(ds2[0])

In [20]:
#CORRIGIR DISTÂNCIA PERCORRIDA ENTRE VIAGENS POSSIVELMENTE COLADAS,SEM RETORNO A PORTO ALEGRE
import datetime
for i in range(1,len(ida)):
    date1 = datetime.datetime.strptime(volta[i-1],'%Y-%m-%d')
    date2 = datetime.datetime.strptime(ida[i],'%Y-%m-%d')
    diff = (date2 - date1).days
    if (diff>0 and diff<2):
        #Emendou viagem com outra: Provavelmente não retornou a Porto Alegre
        #Vamos corrigir a estimativa de distância 
        last_city = destino[i-1].split(',')[-1].lstrip().rstrip()
        first_city = destino[i].split(',')[0].lstrip().rstrip()
           
        distancia[i-1]-=float(get_distance(last_city,"Porto Alegre"))
        distancia[i]-=float(get_distance(first_city,"Porto Alegre"))
        distancia[i]+=float(get_distance(first_city,last_city))

In [21]:
for i in range(len(qtdiarias)):
    if(qtdiarias[i]==0):
        distancia[i]=0

In [22]:
#GERAR E SALVAR DATAFRAME
d = {'requisicao':requisicao,'destino':destino,'ida':ida,'volta':volta,'solicitante':solicitante,'qtdiarias':qtdiarias,'valor':valor,'valtotal':valtotal,'numpassagens':numpassagens,'distancia':distancia}
dff = pd.DataFrame(data=d)
dff.to_csv('2_prepared_data/diarias2.csv',index=False) 

In [36]:
import pickle
with open('cities.pkl','wb') as output:
    pickle.dump(saved_cities,output,pickle.HIGHEST_PROTOCOL)

98.33840820772105

In [6]:
import pickle
with open('cities.pkl','rb') as input:
    saved_cities = pickle.load(input)

In [2]:
for i in range(len(diarias)):
    date1 = diarias.ida[i]
    date2 = diarias.volta[i]
    
    ds1 = date1.split('/')
    ds2 = date2.split('/')
    if (len(ds1)==3):
        diarias.ida[i]=str(ds1[2])+'-'+str(ds1[1])+'-'+str(ds1[0])
    if (len(ds2)==3):
        diarias.volta[i]=str(ds2[2])+'-'+str(ds2[1])+'-'+str(ds2[0])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
distancia=[]